# ARP on MNIST (Tiny) #3
Minimal MNIST classifier using ARP for 5 epochs (requires torchvision).

In [ ]:
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from adaptive_dynamics.arp.optimizers import ARP
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tfm = transforms.Compose([transforms.ToTensor()])
train = datasets.MNIST(root='./data', train=True,  transform=tfm, download=True)
test  = datasets.MNIST(root='./data', train=False, transform=tfm, download=True)
train_loader = DataLoader(train, batch_size=128, shuffle=True)
test_loader  = DataLoader(test,  batch_size=256)

model = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 128), nn.ReLU(), nn.Linear(128,10)).to(device)
opt = ARP(model.parameters(), lr=0.006, alpha=0.01, mu=0.001)
loss_fn = nn.CrossEntropyLoss()

def eval_acc():
    model.eval(); c=0; t=0
    with torch.no_grad():
        for X,y in test_loader:
            X,y = X.to(device), y.to(device)
            p = model(X).argmax(1)
            c += (p==y).sum().item(); t += y.numel()
    return c/t

for ep in range(1, 5+1):
    model.train()
    for X,y in train_loader:
        X,y = X.to(device), y.to(device)
        loss = loss_fn(model(X), y)
        loss.backward(); opt.step(); opt.zero_grad()
    print('epoch', ep, 'acc=', round(eval_acc(),3))